In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from technical_analysis.utils.utils import *

c:\Users\b90361388\Desktop\sst_iteso\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("Notebook/files/Binance_BTCUSDT_1h.csv", header=1)
data.rename(columns={'Volume BTC':'Volume'}, inplace=True)

data.index = pd.to_datetime(data['Date'], format="ISO8601")
data.sort_index(inplace=True)
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume,Volume USDT,tradecount
Date,,,,,,,,,,
2017-08-17 04:00:00,1502942400000,2017-08-17 04:00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,2.023661e+05,171
2017-08-17 05:00:00,1502946000000,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102
2017-08-17 06:00:00,1502949600000,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
2017-08-17 07:00:00,1502953200000,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
2017-08-17 08:00:00,1502956800000,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
...,...,...,...,...,...,...,...,...,...,...
2025-09-22 19:00:00,1758567600000,2025-09-22 19:00:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
2025-09-22 20:00:00,1758571200000,2025-09-22 20:00:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
2025-09-22 21:00:00,1758574800000,2025-09-22 21:00:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931


In [9]:
# --- Ejemplo de uso ---
# Suponiendo que tienes 'data' como tu DataFrame de precios

# 1. Definir el grid de parámetros para la optimización
param_grid = {
    "rsi": {
        'window': [10, 50],
        'buy_threshold': [20, 40],
        'sell_threshold': [60, 80]
    },
    'n_shares': [1, 5]
}

# 2. Definir qué combinaciones de estrategias quieres probar
strategies_to_combine = [['rsi']]

# 3. Dividir los datos y ejecutar la optimización
train_size = int(len(data) * 0.7)
train_df = data.iloc[:train_size]
validation_df = data.iloc[train_size:]

best_params = optimize_hyperparameters(
    param_grid,
    strategies_to_combine,
    train_df,
    validation_df,
    n_trials=50 # Aumentar para una búsqueda más exhaustiva
)

# 4. Construir la mejor estrategia con los parámetros encontrados
best_strat_names = eval(best_params['strategy_combination'])
best_strategy_instances = []
for name in best_strat_names:
    params = {k.split('_', 1)[1]: v for k, v in best_params.items() if k.startswith(name)}
    best_strategy_instances.append(STRATEGY_MAPPING[name](**params))

final_strategy = CompoundStrategy(strategies=best_strategy_instances)
final_n_shares = best_params['n_shares']
final_sl_tp = (
    best_params['sl_long_factor'], 
    best_params['tp_long_factor'],
    1 + (1 - best_params['sl_long_factor']),
    1 - (best_params['tp_long_factor'] - 1)
)


# 5. Ejecutar una validación cruzada final con la mejor estrategia
cv_backtester = CrossValidationBacktester(full_data=data, n_splits=5)
results = cv_backtester.run_cv(
    strategy=final_strategy, 
    n_shares=final_n_shares,
    sl_tp_factors=final_sl_tp
)

# 6. Visualizar los resultados
plot_backtesting_results(results)

[I 2025-10-07 15:24:27,785] A new study created in memory with name: no-name-a72b8ac8-60ff-441c-87e1-e89f91781cc7
[I 2025-10-07 15:24:28,913] Trial 0 finished with value: 361.77207538215157 and parameters: {'strategy_combination': "['rsi']", 'rsi_window': 35, 'rsi_buy_threshold': 24, 'rsi_sell_threshold': 67, 'n_shares': 1, 'sl_long_factor': 0.9878314471176066, 'tp_long_factor': 1.0693309828247048}. Best is trial 0 with value: 361.77207538215157.
[I 2025-10-07 15:24:30,034] Trial 1 finished with value: 10.620861895997177 and parameters: {'strategy_combination': "['rsi']", 'rsi_window': 50, 'rsi_buy_threshold': 27, 'rsi_sell_threshold': 77, 'n_shares': 1, 'sl_long_factor': 0.9743709771993817, 'tp_long_factor': 1.0204024208890805}. Best is trial 0 with value: 361.77207538215157.
[I 2025-10-07 15:24:30,985] Trial 2 finished with value: 284.1965163137308 and parameters: {'strategy_combination': "['rsi']", 'rsi_window': 38, 'rsi_buy_threshold': 23, 'rsi_sell_threshold': 64, 'n_shares': 2, '

Mejores parámetros encontrados: {'strategy_combination': "['rsi']", 'rsi_window': 39, 'rsi_buy_threshold': 22, 'rsi_sell_threshold': 68, 'n_shares': 1, 'sl_long_factor': 0.985143881876558, 'tp_long_factor': 1.0340943108292968}
Ejecutando Fold 1/5...
Ejecutando Fold 2/5...
Ejecutando Fold 3/5...
Ejecutando Fold 4/5...
Ejecutando Fold 5/5...


In [10]:
from performance_metrics.metrics import win_rate, maximum_drawdown

last_fold_idx = max(results.keys())
last_fold_df = results[last_fold_idx]

metrics = {}
for freq_label, freq_str in zip(['monthly', 'quarterly', 'yearly'], ['M', 'Q', 'Y']):
    resampled = last_fold_df['portfolio_value'].resample(freq_str)
    period_values = [group for _, group in resampled]
    win_rates = [win_rate(pv) for pv in period_values]
    drawdowns = [maximum_drawdown(pv) for pv in period_values]
    metrics[freq_label] = {
        'win_rate': win_rates,
        'maximum_drawdown': drawdowns
    }

metrics

{'monthly': {'win_rate': [0.10410958904109589,
   0.0,
   0.06998654104979811,
   0.063257065948856,
   0.0,
   0.11709286675639301,
   0.043115438108484005,
   0.0,
   0.020188425302826378,
   0.0,
   0.010040160642570281,
   0.08208955223880597,
   0.0449438202247191,
   0.059506531204644414,
   0.08710801393728224,
   0.08143322475570032,
   0.0],
  'maximum_drawdown': [0.0,
   0.0,
   -4.95023745386014e-05,
   -6.691563181571158e-05,
   0.0,
   -0.0003811458352338537,
   -0.0006069391353640263,
   0.0,
   -3.642748591670131e-05,
   0.0,
   0.0,
   -0.0003812540420786714,
   -0.0003180686300039426,
   -9.774640837689752e-05,
   0.0,
   0.0,
   0.0]},
 'quarterly': {'win_rate': [0.035023041474654376,
   0.04485727231536022,
   0.05346624376982329,
   0.009380863039399626,
   0.06034038164002063,
   0.058997050147492625],
  'maximum_drawdown': [0.0,
   -4.95023745386014e-05,
   -0.0003811458352338537,
   -3.642748591670131e-05,
   -0.0003812540420786714,
   0.0]},
 'yearly': {'win_rat

In [11]:
monthly = pd.DataFrame(metrics['monthly'])
quarterly = pd.DataFrame(metrics['quarterly'])
yearly = pd.DataFrame(metrics['yearly'])

In [17]:
yearly

,win_rate,maximum_drawdown
0,0.046355,0.000000
1,0.038407,-0.000036
